# **Laboratorio 11: Pienso, luego predigo 💡**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Felipe Hernández M.
- Nombre de alumno 2: Brandon Peña H.

### **Link de repositorio de GitHub:** [Enlace](https://github.com/brandonHaipas/MDS7202-Lab-Prog-Ciencia-de-Datos)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [8]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq "gymnasium[box2d]"

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [1]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

El ambiente de `blackjack` contiene la implementación del juego de Blackjack como un problema de RL. El objetivo del juego es ganarle al _dealer_ consiguiendo cartas cuyo puntaje sume un valor más cercano a 21 (sin superar el 21) que las cartas del _dealer_.

El puntaje de cada carta se describe a continuación:
- Los _ases_ pueden contar como 11 o 1.
- Las cartas J, Q, K tienen un valor de 10.
- Las cartas numéricas (2 a 10) tienen un valor equivalente a su número.

El juego comienza con el _dealer_ sacando dos cartas: una visible y la otra boca abajo.

En cada paso del juego, el jugador puede solicitar _(hit)_ una nueva carta hasta que decida quedarse _(stick)_ con las que posee actualmente. Si al solicitar una nueva carta el jugador supera el valor 21, pierde automáticamente.

Una vez el jugador decide quedarse, el _dealer_ revela su carta boca abajo y saca nuevas cartas hasta que la suma de sus puntos sea 17 o mayor, perdiendo si llega a superar el valor 21.

La formulación del problema como un _Markov Decision Process_ consiste de los siguientes componentes:
- Estados: Cada observación consiste de una 3-tupla $(p, d, a)$, donde $p$ corresponde a la suma de puntos actual del jugador, $d$ el puntaje de la carta visible del _dealer_ (un valor de 1 a 10, donde 1 es un _as_) y $a$ un valor 1 o 0 indicando si el jugador posee o no una carta _as_, respectivamente. Cada uno de estos valores es un número entero.
- Acciones: El espacio de acción es de dimensión 1, cuyos posibles valores son:
    - 0: Quedarse (stick)
    - 1: Solicitar una nueva carta (hit)
- Recompensas:
    - Ganar el juego: +1 o +1.5 si es un _natural blackjack_ (ganar por comenzar con un _as_ y un 10, el uso del puntaje adicional para este caso es opcional y requiere el uso del argumento `natural=True`)
    - Perder el juego: -1
    - Empatar el juego: 0

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [ ]:
import random
import numpy as np

random.seed(0)

n_simulations = 5000
action_space = [0, 1]
rewards = []

for episode in range(n_simulations):
    env.reset(seed=random.randint(1, 10000))
    end = False

    while not end:
        action = random.choice(action_space)
        state, reward, done, truncated, info = env.step(action)
        end = done | truncated
    rewards.append(reward)

print('Recompensa promedio:', np.mean(rewards))
print('Desviación estándar de recompensas:', np.std(rewards))

Recompensa promedio: -0.3866
Desviación estándar de recompensas: 0.8999669105028251


La performance de esta política aleatoria no es buena, en promedio el agente tiene una recompensa negativa de -0.3866, es decir, pierde 2/3 de las veces que no empata.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [3]:
from stable_baselines3 import DQN

# init agent
model = DQN("MlpPolicy", env, verbose=0)

# train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)

Output()

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [4]:
n_simulations = 5000
rewards = []

for episode in range(n_simulations):
    end = False
    result = None
    state, info = env.reset(seed=random.randint(1, 10000))

    while not end:
        action, _states = model.predict(state, deterministic=True)
        action = action.item()
        state, reward, done, truncated, info = env.step(action)
        end = done | truncated
    rewards.append(reward)

print('Recompensa promedio:', np.mean(rewards))
print('Desviación estándar de recompensas:', np.std(rewards))

Recompensa promedio: -0.054
Desviación estándar de recompensas: 0.9458773704873165


La recompensa promedio es mayor y más cercana a 0 y hay más dispersión en estas. Luego, se observa que el agente está actuando con más racionalidad, y tiene mejor performance que el azar.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [5]:
state, info = env.reset()
state

array([21,  2,  1])

Los estados son de clase `np.array`.

In [6]:
def get_action(state):
    action, _states = model.predict(state, deterministic=True)
    action = action.item()

    if action == 1:
        return 'Hit'
    else:
        return 'Stick'

# Definición de estados
s1 = np.array([6, 7, 0])
s2 = np.array([19, 3, 1])

print("Estado: Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as\nAcción:", get_action(s1))
print("Estado: Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as\nAcción:", get_action(s2))

Estado: Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
Acción: Hit
Estado: Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as
Acción: Stick


Las acciones tienen sentido. En el primer estado, el agente tiene una suma de 6 y sin as, está lejos de 21 y es mejor realizar un "hit" para pedir otra carta. En el segundo caso, tiene una suma de 19 y un as, está muy cerca de 21. No tiene sentido pedir otra carta, por lo que decide un "stick" para quedarse.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [10]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [12]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

El ambiente `Lunar Lander` contiene un problema de optimización de la trayectoria de un cohete en 2D, con el objetivo de aterrizar en una plataforma en la posición (0,0), planteado como un problema de RL.

En cada episodio, el cohete debe decidir una secuencia de movimientos de motor y propulsores (encenderlos o apagarlos) para aterrizar correctamente. El episodio termina si:

- El cohete choca (su cuerpo entra en contacto con la superficie).
- El cohete sale de la vista (coordenada $x$ con magnitud mayor a 1).
- El cohete está en reposo.

Su formulación en MDP consiste de los siguientes componentes (tomando en consideración el uso del parámetro `continuous=True`:

- Estados: Cada observación corresponde a un vector de 8 dimensiones, con las coordenadas de la posición del cohete ($x$ e $y$ ambos con valores entre -2.5 y +2.5), sus velocidades lineales en $x$ e $y$ (ambas de -10 a +10), su ángulo de inclinación (de $-2\pi$ a $+2\pi$), su velocidad angular (de -10 a +10) y dos valores `booleans` que representan si cada "pata" del cohete están en contacto con la superficie o no.

- Acciones: A diferencia del ambiente de `blackjack`, el espacio de acción es continuo, conformado por dos coordenadas (`main`, `lateral`) que pueden tomar valores de -1 a +1. La primera coordenada `main` indica la aceleración del motor principal del cohete, estándo completamente apagado cuando `main < 0` y escala de forma lineal entre 50% y 100% cuando `0 <= main <= 1`. De manera similar, la coordenada `lateral` indica la aceleración de los propulsores laterales, donde estarán ambos apagados si `-0.5 < lateral < 0.5` y escalará la propulsión linealmente entre 50% y 100% si `-1 <= lateral <= -0.5` (o `0.5 <= lateral <= 1`) para el propulsor izquierdo (o derecho, respectivamente).

- Recompensas: A diferencia del ambiente de `blackjack` (donde la recompensa se obtiene sólo al terminar el juego) hay una recompensa después de cada paso, obteniendose una recompensa total del espisodio como la suma de las obtenidas en cada paso de este.

    Para cada paso, la recompensa:

    - Aumenta/disminuye según lo cerca/lejos se encuentre el cohete de la plataforma de aterrizaje.
    - Aumenta/disminuye según lo lento/rápido se mueva el cohete.
    - Disminuye mientras más inclinado esté el cohete.
    - Aumenta en 10 puntos por cada "pata" en contacto con la superficie.
    - Disminuye en 0.03 puntos por cada "cuadro" de la trayectoria donde un propulsor esté encendido.
    - Disminuye en 0.3 puntos por cada "cuadro" de la trayectoria donde el motor principal esté encendido.

    Un episodio recibe una recompensa adicional en los siguientes casos:

    - Aumenta 100 puntos si aterriza de forma segura.
    - Disminuye 100 puntos si choca.

    Finalmente, un episodio se considera una solución válida si logra al menos 100 puntos.

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [13]:
import random
import numpy as np

n_simulations = 10
rewards = []

for episode in range(n_simulations):
    env.reset(seed=random.randint(1, 10000))
    end = False

    while not end:
        main = random.uniform(-1, 1)
        lateral = random.uniform(-1, 1)
        state, reward, done, truncated, info = env.step([main, lateral])
        end = done | truncated
    rewards.append(reward)

print('Recompensa promedio:', np.mean(rewards))
print('Desviación estándar de recompensas:', np.std(rewards))

Recompensa promedio: -100.0
Desviación estándar de recompensas: 0.0


La performance de esta política es mala, en promedio la recompensa es de -100, lo que indica que el cohete siempre choca.

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [14]:
from stable_baselines3 import PPO

# init agent
model = PPO("MlpPolicy", env, verbose=0)

# train the agent and display a progress bar
model.learn(total_timesteps=10000, progress_bar=True)

Output()

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [15]:
n_simulations = 10
rewards = []

for episode in range(n_simulations):
    end = False
    result = None
    state, info = env.reset(seed=random.randint(1, 10000))

    while not end:
        action, _states = model.predict(state, deterministic=True)
        state, reward, done, truncated, info = env.step(action)
        end = done | truncated
    rewards.append(reward)

print('Recompensa promedio:', np.mean(rewards))
print('Desviación estándar de recompensas:', np.std(rewards))

Recompensa promedio: -100.0
Desviación estándar de recompensas: 0.0


La performance sigue siendo igual de mala que el caso baseline.

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [24]:
model2 = PPO("MlpPolicy", env, verbose=0)

# train the agent and display a progress bar
model2.learn(total_timesteps=200000, progress_bar=True)

Output()

In [25]:
n_simulations = 10
rewards = []

for episode in range(n_simulations):
    end = False
    result = None
    state, info = env.reset(seed=random.randint(1, 10000))

    while not end:
        action, _states = model2.predict(state, deterministic=True)
        state, reward, done, truncated, info = env.step(action)
        end = done | truncated
    rewards.append(reward)

print('Recompensa promedio:', np.mean(rewards))
print('Desviación estándar de recompensas:', np.std(rewards))

Recompensa promedio: 80.0
Desviación estándar de recompensas: 60.0


In [26]:
export_gif(model2)

Al entrenar con 200000 timesteps el agente obtiene una recompensa promedio de 80. A continuación se puede visualizar el gif generado, donde se aprecia que el agente aterriza de forma cuidadosa, casi dentro de la plataforma.

<p align="center">
  <img src="agent_performance.gif"
" width="400">
</p>

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [19]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [20]:
%pip install --upgrade --quiet PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [21]:
import PyPDF2

doc_paths = ["coli_a_00524.pdf", "2409.16430v1.pdf"] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [22]:
%pip install --upgrade --quiet langchain-google-genai faiss-cpu langchain_community pypdf wikipedia

Note: you may need to restart the kernel to use updated packages.


In [27]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

def generate_vectorstore(doc_paths=doc_paths, chunk_size=500, chunk_overlap=50):
    docs = list()

    # Load
    for doc in doc_paths:
        loader = PyPDFLoader(doc)

        docs += loader.load()

    # Split
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap) # inicializamos splitter
    splits = text_splitter.split_documents(docs) # dividir documentos en chunks

    # Embed & Store
    embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
    vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
    return vectorstore

base_vectorstore = generate_vectorstore()

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [58]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", # modelo de lenguaje
    temperature=0, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

# Función auxiliar para formatear respuesta
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def generate_rag_chain(vectorstore=base_vectorstore, k=3, search_type="similarity"):
    # Retriever chain
    retriever = vectorstore.as_retriever(search_type=search_type, # método de búsqueda
                                        search_kwargs={"k": k}, # n° documentos a recuperar
                                        )
    retriever_chain = retriever | format_docs # chain

    # RAG chain
    llm_bias_rag_template = '''
    You are an expert assistant specializing in biases in large language models (LLMs).
    Your sole role is to answer user questions strictly based on the relevant information provided to you.
    Always provide the most comprehensive and precise response possible, using only the supplied context.
    Answer only what is asked; NEVER fabricate or infer details beyond the given information.

    Relevant information: {context}
    Question: {question}
    Answer:
    '''

    rag_prompt = PromptTemplate.from_template(llm_bias_rag_template)

    rag_chain = (
        {
            "context": retriever_chain, # context lo obtendremos del retriever_chain
            "question": RunnablePassthrough(), # question pasará directo hacia el prompt
        }
        | rag_prompt # prompt con las variables question y context
        | llm # llm recibe el prompt y responde
        | StrOutputParser() # recuperamos sólo la respuesta
    )

    return rag_chain

base_rag_chain = generate_rag_chain()

#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

Se definen las siguientes tuplas:
- Tupla 1:
    - Pregunta: What are social biases?
    - Respuesta correcta: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
- Tupla 2:
    - Pregunta: Which word embedding-based metrics exist for measuring bias in LLMs?
    - Respuesta correcta: Word Embedding metrics and Sentence Embedding metrics
- Tupla 3:
    - Pregunta: What types of techniques exist for Bias Mitigation in LLMs that intervene in the pre-procesing stage?
    - Respuesta correcta: Data Augmentation, Data Filtering & Reweighting, Data Generation, Instruction Tuning and Projection-based Mitigation


In [86]:
questions = [
    ("What are social biases?", "Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries"),
    ("Which embedding-based metrics exists for measuring bias in LLMs?", "Word Embedding metrics and Sentence Embedding metrics"),
    ("What types of techniques exist for Bias Mitigation in LLMs that intervene in the pre-procesing stage?", "Data Augmentation, Data Filtering & Reweighting, Data Generation, Instruction Tuning and Projection-based Mitigation")
]

def ask_questions(chain=base_rag_chain):
    for q in questions:
        print("Question: " + q[0] + '\nExpected Answer: ' + q[1] + '\nAnswer: ' + chain.invoke(q[0]) + "\n")

ask_questions()

Question: What are social biases?
Expected Answer: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
Answer: Social biases are defined as disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries. Within the LLM system, social bias reflects societal prejudices and stereotypes present in training data.

Question: Which embedding-based metrics exists for measuring bias in LLMs?
Expected Answer: Word Embedding metrics and Sentence Embedding metrics
Answer: The provided information describes what embedding-based metrics are and what they use, but it does not list specific names of embedding-based metrics that exist for measuring bias in LLMs. It mentions "Word Embedding Metrics" as a category and states that one relevant method for static word embeddings is presented, but does not name it.

Question: What types of techniques exist for Bias Mitigation in 

El RAG entrega una buena respuesta para las preguntas 1 y la 3, pero la tupla 2 es menos precisa.

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [87]:
# Cambio de tamaño de chunk
vectorstore_small_chunk = generate_vectorstore(chunk_size=50)
rag_chain_small_chunk = generate_rag_chain(vectorstore=vectorstore_small_chunk)
ask_questions(chain=rag_chain_small_chunk)

Question: What are social biases?
Expected Answer: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
Answer: Based on the provided information, social bias is described as "a subjective and" (the description is incomplete in the provided text). Racial bias is also stated to be a form of social bias.

Question: Which embedding-based metrics exists for measuring bias in LLMs?
Expected Answer: Word Embedding metrics and Sentence Embedding metrics
Answer: The provided information states that "Bias in the embedding space can have a" in the context of metrics for assessing bias in LLMs. However, it does not list any specific embedding-based metrics.

Question: What types of techniques exist for Bias Mitigation in LLMs that intervene in the pre-procesing stage?
Expected Answer: Data Augmentation, Data Filtering & Reweighting, Data Generation, Instruction Tuning and Projection-based Mitigation
Answer: Based on th

In [88]:
# Cambio de tamaño de chunk
vectorstore_big_chunk = generate_vectorstore(chunk_size=1000)
rag_chain_big_chunk = generate_rag_chain(vectorstore=vectorstore_big_chunk)
ask_questions(chain=rag_chain_big_chunk)

Question: What are social biases?
Expected Answer: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
Answer: Based on the provided information, "social bias" is referred to as "bias" unless otherwise specified, and its definition is stated to be in "Definition 7." However, "Definition 7" is not included in the provided text. Therefore, the specific definition of social bias is not available in the given context.

Question: Which embedding-based metrics exists for measuring bias in LLMs?
Expected Answer: Word Embedding metrics and Sentence Embedding metrics
Answer: Embedding-based metrics for measuring bias in LLMs include Word Embedding Metrics. These metrics typically compute distances in the vector space between neutral words (e.g., professions) and identity-related words (e.g., gender pronouns). While first proposed for static word embeddings, their basic formulation of computing cosine distances betwe

Al utilizar chunks más pequeños, el texto parece ser insuficiente para responder preguntas complejas que antes si podía resolver, como la tupla 3. Por otro lado, al usar chunks más grandes el texto proporcionado es de mayor tamaño, y algunas respuestas son más complejas, como la de la tupla 2.

In [89]:
# Cambio de cantidad de chunk recuperados
rag_chain_less_chunks = generate_rag_chain(k=1)
ask_questions(chain=rag_chain_less_chunks)

Question: What are social biases?
Expected Answer: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
Answer: Social biases are defined as disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries.

Question: Which embedding-based metrics exists for measuring bias in LLMs?
Expected Answer: Word Embedding metrics and Sentence Embedding metrics
Answer: Based on the provided information, the text defines embedding-based metrics as those that use dense vector representations, typically contextual sentence embeddings, to measure bias. However, it does not list or name any specific embedding-based metrics.

Question: What types of techniques exist for Bias Mitigation in LLMs that intervene in the pre-procesing stage?
Expected Answer: Data Augmentation, Data Filtering & Reweighting, Data Generation, Instruction Tuning and Projection-based Mitigation
Answer: 

In [90]:
# Cambio de cantidad de chunk recuperados
rag_chain_more_chunks = generate_rag_chain(k=20)
ask_questions(chain=rag_chain_more_chunks)

Question: What are social biases?
Expected Answer: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
Answer: Social bias broadly encompasses disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries. It is a subjective and normative term used to refer to harms such as stereotypes, misrepresentations, derogatory and exclusionary language, and other denigrating behaviors that disproportionately affect already-vulnerable and marginalized communities.

Question: Which embedding-based metrics exists for measuring bias in LLMs?
Expected Answer: Word Embedding metrics and Sentence Embedding metrics
Answer: Based on the provided information, the text describes how embedding-based metrics work (e.g., computing distances in vector space between neutral words and identity-related words, or using cosine similarity to compare words like "doctor" to social group t

Al recuperar pocos chunks (y por ello, menos texto), las respuestas son más dificiles de responder, como en la pregunta 2, donde se indica que el texto es insuficiente. Mientras que muchos chunks de texto aportan más información encontrada, permitiendo construir respuestas más complejas.

In [91]:
# Cambio de tipo de busqueda
rag_chain_other_search = generate_rag_chain(search_type="mmr")
ask_questions(chain=rag_chain_other_search)

Question: What are social biases?
Expected Answer: Social biases are disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries
Answer: Social bias refers to disparate treatment or outcomes between social groups that arise from historical and structural power asymmetries.

Question: Which embedding-based metrics exists for measuring bias in LLMs?
Expected Answer: Word Embedding metrics and Sentence Embedding metrics
Answer: Based on the provided information, the text defines embedding-based metrics as those that use dense vector representations, typically contextual sentence embeddings, to measure bias. However, it does not list any specific named embedding-based metrics for measuring bias in LLMs. The "EMBEDDINGS" section in Table 6 refers to objective functions for bias *mitigation* by modifying embeddings, not metrics for measuring bias.

Question: What types of techniques exist for Bias Mitigation in LLMs that intervene in the 

Finalmente, al cambiar la función de búsqueda la tupla 2 incorpora información más información, pero sin lograr responder correctamente, mientras que la tupla 1 entrega una respuesta casi perfecta y la 3 se mantiene constante.

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [46]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results = 1)

/tmp/ipykernel_31006/4138654522.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results = 1)


#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [47]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wikipedia_query = WikipediaQueryRun(api_wrapper=api_wrapper)

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [61]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

tools = [tavily_search, wikipedia_query]
react_prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, react_prompt) # primero inicializamos el agente ReAct
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # lo transformamos a AgentExecutor para habilitar la ejecución de tools

response = agent_executor.invoke({"input": "Hola, cual es la capital de chile?"})
print(response["output"])

/home/felipe/.pyenv/versions/lab11/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: capital de chilePage: Santiago
Summary: Santiago (, US also ; Spanish: [sanˈtjaɣo]), also known as Santiago de ChileI now know the final answer
Final Answer: La capital de Chile es Santiago.

> Finished chain.
La capital de Chile es Santiago.


El agente responde en español si problemas.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

El agente debería usar la tool de Tavily cuando la pregunta es sobre poco precisa respecto al tópico, como al consultar "Cómo puedo cocinar un pollo?"

In [62]:
response = agent_executor.invoke({"input": "Cómo puedo cocinar un pollo?"})
print(response["output"])



> Entering new AgentExecutor chain...
Action: tavily_search_results_json
Action Input: cómo cocinar un pollo[{'title': 'Pollo en salsa. Receta tradicional y casera', 'url': 'https://recetasdecocina.elmundo.es/2024/10/pollo-en-salsa-receta-tradicional-y-casera.html', 'content': 'Hacer pollo en salsa ... 1.- Comenzamos salpimentando los trozos de pollo y dorándolos a fuego fuerte en una cocerlo. Una vez dorados, los', 'score': 0.523494}]Action: tavily_search_results_json
Action Input: métodos para cocinar pollo[{'title': 'Cómo cocinar las distintas partes del pollo - Recetas Nestlé', 'url': 'https://www.recetasnestlecam.com/escuela-sabor/recetas/preparaciones-pollo', 'content': 'La mejor pechuga de pollo · A la plancha: este método se suele usar con los filetes, aprovechando que son más delgados. · En brochetas: aprovecha', 'score': 0.6342494}]Action: tavily_search_results_json
Action Input: formas comunes de cocinar pollo[{'title': 'Las 71 mejores recetas de pollo - Directo al Paladar

Mientras que la tool de Wikipedia debe emplearse para responder preguntas sobre un tópico específico, como personas, lugares, eventos históricos, etc.

In [64]:
response = agent_executor.invoke({"input": "De qué nacionalidad es Lionel Messi?"})
print(response["output"])



> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: Lionel MessiPage: Lionel Messi
Summary: Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesiThought: The full summary from Wikipedia clearly states that Lionel Messi "is an Argentine professional footballer" and "captains the Argentina national team." It also mentions he "left his home country of Argentina".
Final Answer: Lionel Messi es de nacionalidad argentina.

> Finished chain.
Lionel Messi es de nacionalidad argentina.


### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [65]:
from langchain.tools import tool

@tool
def search_bias_query(query: str) -> str:
    """Asks a query to an expert about Biases in LLMs."""
    return base_rag_chain.invoke(query)

@tool
def search_general_query(query: str) -> str:
    """Asks a query about a general topic."""
    return agent_executor.invoke({"input": query})["output"]

multiagent_tools = [search_bias_query, search_general_query]

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [67]:
supervisor_agent = create_react_agent(llm, multiagent_tools, react_prompt) # primero inicializamos el agente ReAct
supervisor = AgentExecutor(agent=supervisor_agent, tools=multiagent_tools, verbose=True)

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [93]:
import time
for q in questions:
    response = supervisor.invoke({"input": q[0]})
    print("Question:", q[0], "\nAnswer:", response["output"])
    time.sleep(30)  # Para prevenir que se caiga por la cuota del free tier de Gemini



> Entering new AgentExecutor chain...
Action: search_general_query
Action Input: What are social biases?

> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: social biasesPage: List of cognitive biases
Summary: Cognitive biases are systematic patterns of deviation from nAction: wikipedia
Action Input: social biasPage: Social-desirability bias
Summary: In social science research social-desirability bias is a typAction: tavily_search_results_json
Action Input: what are social biases definition[{'title': 'Social Biases | Free Essay Example for Students - Aithor', 'url': 'https://aithor.com/essay-examples/social-biases', 'content': 'A social bias is a process in which social groups can be categorized, often in a stereotypical way, and are treated differently and in an unfair manner as a result. There are many forms of social bias, including the most well-known form, which is prejudice. Prejudice is an unjust and usually negative attitude towards a group and its individu

In [70]:
response = supervisor.invoke({"input": "Cómo puedo cocinar un pollo?"})
print(response["output"])



> Entering new AgentExecutor chain...
Action: search_general_query
Action Input: Cómo cocinar un pollo

> Entering new AgentExecutor chain...
Action: tavily_search_results_json
Action Input: Cómo cocinar un pollo[{'title': 'Pollo - 74,888 recetas caseras- Cookpad', 'url': 'https://cookpad.com/eeuu/buscar/pollo', 'content': '*   \n[Pollo en cazuela](https://cookpad.com/eeuu/recetas/17014457)\n-------------------------------------------------------------  Guarda esta receta para encontrarla más fácilmente cuando la quieras cocinar.   [](https://cookpad.com/eeuu/buscar/pollo# "Cerrar")          muslos de pollo • Sal • Pimienta • Ajo • Jugo de un limón o naranja agria • Curry • Romero • Vino seco • Aceite vegetal • Puré de tomate   \n\n    *   1h y 30 min\n    *   3 o 4 raciones [...] ------------------------------------------------------------------------------------------------------------------------------------  Guarda esta receta para encontrarla más fácilmente cuando la quieras coc

In [71]:
response = supervisor.invoke({"input": "De qué nacionalidad es Lionel Messi?"})
print(response["output"])



> Entering new AgentExecutor chain...
Action: search_general_query
Action Input: What is Lionel Messi's nationality?

> Entering new AgentExecutor chain...
Action: wikipedia
Action Input: Lionel MessiPage: Lionel Messi
Summary: Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesiThought: The user is asking for Lionel Messi's nationality. I have used the Wikipedia tool and received a snippet of the summary. While the snippet itself does not explicitly state his nationality, it is common knowledge and typically the first piece of information in a Wikipedia summary for a person. I will assume that the full summary from Wikipedia would contain this information, and based on general knowledge, his nationality is Argentinian.
Final Answer: Lionel Messi's nationality is Argentinian.

> Finished chain.
Lionel Messi's nationality is Argentinian.Final Answer: Es argentino.

> Finished chain.
Es argentino.


Las respuestas del enfoque 2.1.4 cambian dado que la pregunta 2 no se responde con la herramienta de RAG, sino que al ser una pregunta más general se consulta con la herramienta del agente. Además, ahora que el agente de RAG no responde directamente, el agente supervisor utiliza su respuesta y la entrega en una redacción distinta. Por otro lado, las respuestas del enfoque 2.2.4 utilizan la herramienta del agente, como se esperaba, y entrega respuestas similares a las generadas previamente.

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

La diferencia es que el agente, en vez de decidir una "categoría" que después es parseada para llamar a un segundo agente que resuelve la pregunta, usa a los agentes como herramientas. Una ventaja de esto es que puede llamar a ambos agentes en una sola ejecución si lo considera pertinente, mientras que una desventaja es que realiza más ejecuciones del LLM, porque ahora el supervisor también tiene un template tipo ReAct.

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

In [75]:
# Extendemos el prompt de React para incorporar la interacción previa
memory_prompt = hub.pull("hwchase17/react")
memory_prompt.template = 'Previous interaction: {memory}\n' + memory_prompt.template
memory_prompt.input_variables.append('memory')

memory_agent = create_react_agent(llm, multiagent_tools, memory_prompt) # primero inicializamos el agente ReAct
memory_agent_executor = AgentExecutor(agent=memory_agent, tools=multiagent_tools, verbose=True)

/home/felipe/.pyenv/versions/lab11/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [76]:
memory = ''

def invoke_agent_with_memory(input, memory):
    response = memory_agent_executor.invoke({"input": input, "memory": memory})
    memory += f'User: {input}\n Agent:{response["output"]}'
    print(response["output"])
    return memory

memory = invoke_agent_with_memory("Hola, soy Felipe!", memory)



> Entering new AgentExecutor chain...
Final Answer: ¡Hola Felipe! Mucho gusto.

> Finished chain.
¡Hola Felipe! Mucho gusto.


In [79]:
memory = invoke_agent_with_memory("Cuál es mi nombre?", memory)



> Entering new AgentExecutor chain...
Final Answer: Tu nombre es Felipe.

> Finished chain.
Tu nombre es Felipe.


### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [73]:
%pip install --upgrade --quiet gradio

Note: you may need to restart the kernel to use updated packages.


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [ ]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = memory_agent_executor.invoke({"input": message, "memory": history})["output"] # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://6934e35b6a75d4f859.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Final Answer: ¡Hola! ¿En qué puedo ayudarte hoy?

> Finished chain.


> Entering new AgentExecutor chain...
Final Answer: ¡Hola, Felipe! Es un placer conocerte. ¿Hay algo en lo que pueda ayudarte hoy?

> Finished chain.


> Entering new AgentExecutor chain...
Final Answer: Te llamas Felipe.

> Finished chain.


# Conclusión
Éxito!
<center>
<img src ="https://media.tenor.com/MRQgxcelAV8AAAAM/perry-the-platypus-phineas-and-ferb.gif" width = 400 />